In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler


2023-01-27 11:38:14,924 [12261] WARNING  py.warnings:109: [JupyterRequire] /Users/javi/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



In [2]:
auth_manager = SpotifyClientCredentials('74c040f7ab8049648ec083fdadfc1ddb','a9fa947f1cde460bb37b94060380a6cb')
sp = spotipy.Spotify(auth_manager=auth_manager)

In [3]:
playlist_16_dec = 'https://open.spotify.com/playlist/37i9dQZF1DX4JAvHpjipBk'

In [4]:
def spoti_info(playlist):
    play = sp.playlist_tracks(playlist, fields=None, limit=100,\
                          offset=0, market=None, additional_types=('track', 'episode'))
    links_track = []
    for e in (play['items']):
        links_track.append(e['track']['external_urls']['spotify'])
    
    audio_features = []
    for e in links_track:
        audio_features.append(sp.audio_features(e))
    
    track_song = []
    for e in (play['items']):
        track_song.append(e['track']['name'])
    
    artist_name = []
    for e in (play['items']):
        artist_name.append(e['track']['album']['artists'][0]['name'])
    
    audio_features=[e[0] for e in audio_features]
    
    datos_spotipy = pd.DataFrame(audio_features)
    
    datos_spotipy['artist_name'] = artist_name
    
    datos_spotipy['track_name'] = track_song
    
    datos_spotipy.drop(['type','id','uri','track_href','analysis_url','time_signature','key','mode'],axis=1, inplace=True)
    
    datos_spotipy=datos_spotipy.iloc[:, [10,11,1,0,2,4,3,6,7,8,9]]
    
    datos_spotipy.rename(columns = {'duration_ms':'duration'}, inplace = True)
    
    datos_spotipy['duration'] = (datos_spotipy['duration']/1000).round(2)





    return datos_spotipy

In [5]:
new_songs = spoti_info(playlist_16_dec)

In [6]:
new_songs

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration
0,Sam Smith,I’m Not Here To Make Friends (feat. Calvin Har...,0.895,0.698,-4.439,0.17200,0.0628,0.4090,0.838,114.914,229.60
1,ROSALÍA,LLYLM,0.634,0.555,-5.522,0.12900,0.2730,0.1880,0.555,170.228,174.04
2,The Kid LAROI,Love Again,0.398,0.662,-6.691,0.71600,0.0275,0.1110,0.468,107.001,145.85
3,Ava Max,Ghost,0.931,0.715,-3.548,0.00573,0.0426,0.0549,0.615,119.971,181.65
4,Cordae,Two Tens (feat. Anderson .Paak),0.546,0.603,-7.525,0.19400,0.3500,0.0783,0.667,87.990,152.37
...,...,...,...,...,...,...,...,...,...,...,...
95,Ray Vaughn,Sandcastles,0.701,0.802,-5.564,0.03410,0.2140,0.2050,0.476,135.045,170.66
96,ghost orchard,all in,0.498,0.716,-6.643,0.71900,0.1560,0.1070,0.521,84.252,83.13
97,Prentiss,i tried,0.568,0.399,-9.813,0.42800,0.1190,0.2440,0.265,119.978,220.04
98,Red Leather,DAKOTA,0.723,0.626,-4.464,0.14800,0.0356,0.0695,0.592,173.016,166.22


In [ ]:
datos_spotipy_sp.to_csv('../CSV_full/datos_proyecto_final.csv', index=False)

In [7]:
new_songs = new_songs.copy()

In [8]:
new_songs_norm = new_songs.copy()

In [9]:
new_songs_norm['artist_name']=LabelEncoder().fit_transform(new_songs_norm['artist_name'])


In [10]:
scaler=StandardScaler()

In [11]:
new_songs_norm[['energy','danceability','loudness','acousticness','speechiness','liveness',\
                  'valence','tempo','duration',
                    'artist_name']] = scaler.fit_transform(new_songs_norm[['energy',\
                    'danceability','loudness','acousticness','speechiness','liveness',\
                  'valence','tempo','duration','artist_name']])

In [12]:
new_songs_norm.drop(['track_name'],axis=1,inplace=True)

In [13]:
new_songs_norm

,artist_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration
0,0.610074,1.107956,0.458115,0.740934,-0.123326,-0.390903,1.839740,1.511719,-0.242794,0.767498
1,0.218773,-0.220254,-0.564048,0.304764,-0.294342,1.922056,0.126580,0.204990,1.636687,-0.436634
2,0.930230,-1.421240,0.200787,-0.166043,2.040215,-0.779331,-0.470313,-0.196725,-0.511665,-1.047586
3,-1.631014,1.291157,0.579631,1.099778,-0.784599,-0.613176,-0.905192,0.482035,-0.070966,-0.271706
4,-1.132995,-0.668079,-0.220945,-0.501930,-0.035830,2.769334,-0.723798,0.722141,-1.157629,-0.906280
...,...,...,...,...,...,...,...,...,...,...
95,0.325491,0.120704,1.201507,0.287848,-0.671768,1.272843,0.258362,-0.159786,0.441225,-0.509888
96,1.463822,-0.912348,0.586779,-0.146711,2.052146,0.634633,-0.501320,0.047998,-1.284640,-2.406894
97,0.112054,-0.556123,-1.679136,-1.423406,0.894811,0.227500,0.560684,-1.134061,-0.070728,0.560307
98,0.361064,0.232661,-0.056541,0.730866,-0.218777,-0.690202,-0.792015,0.375835,1.731419,-0.606114


In [16]:
new_songs_norm.to_csv('../../Hackshow/new_songs_norm.csv', index=False)

In [18]:
new_songs.to_csv('../../Hackshow/new_songs.csv', index=False)

In [15]:
new_songs

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration
0,Simon Franglen,Nothing Is Lost (You Give Me Strength),0.693,0.520,-7.704,0.00020,0.0331,0.1220,0.120,126.985,267.50
1,Juice WRLD,Face 2 Face,0.720,0.711,-4.532,0.09480,0.0308,0.1010,0.437,155.997,124.56
2,ROSALÍA,DESPECHÁ RMX,0.582,0.954,-5.744,0.23500,0.0727,0.0572,0.884,130.008,170.83
3,Only The Family,Mad Max,0.640,0.898,-8.796,0.02120,0.2550,0.0919,0.680,149.988,153.60
4,Little Simz,Gorilla,0.813,0.716,-5.748,0.26900,0.2560,0.1040,0.682,90.418,245.82
...,...,...,...,...,...,...,...,...,...,...,...
95,young friend,little pieces,0.576,0.564,-9.626,0.21700,0.1310,0.6740,0.677,79.896,157.50
96,chemical club,Bounds,0.713,0.531,-7.337,0.00118,0.0324,0.3280,0.692,150.043,207.07
97,whiterosemoxie,whts rght,0.509,0.444,-9.560,0.34500,0.0881,0.1040,0.237,113.780,258.00
98,Mokita,High,0.570,0.669,-6.364,0.14400,0.0558,0.2660,0.299,75.029,166.83
